## Common functions and prompts

Before you start, 
-  create a python venv, and `pip install -r requirements.txt`.
-  make a copy of `local.env` to `.env`, configure your environment in `.env`.
-  adjust data in the `data` folder based on your configuration.
-  run each scripts in the `ingest` folder to process and ingest the data.

In [ ]:

import sys
import json
import pprint
from openai.types.chat import ChatCompletionMessage

sys.path.append('./utils')
from loadenv import LLMConfig, RobotConfig
sys.path.append('./chat/functions')
from robot_general import get_robots, search_robot_manuals

aoai_config = LLMConfig()
aoai_api_version, aoai_deployment = aoai_config.chat["api_version"], aoai_config.chat['deployment']

def print_llm_messages(msg):
  print("-----------------------------------LLM message(s)---------------------------------------------------")
  pp = pprint.PrettyPrinter(width=160)
  if isinstance(msg, list):
    for m in msg:
      if isinstance(m, dict):
        pp.pprint(m)
      elif isinstance(m, ChatCompletionMessage):
        pp.pprint(dict(m))
  if isinstance(msg, ChatCompletionMessage):
    pp.pprint(dict(msg))
  

In [ ]:
a=get_robots("petoi_cat")
pprint.pprint(json.loads(a))


In [ ]:
b=search_robot_manuals("api for petoi cat")
pprint.pprint(json.loads(b))

In [ ]:
system_message = {"role": "system", "content": 
    """You are an expert who helps people troubleshoot robot related issues.
    A robot's name is typically prefixed with its kind, for example, the kind for spot_jr is spot.
    Try identify the kind of the robot first so you can narrow down when searching for info in tools. 
    Respond only with the info from the provided context or tools.
    If you don't know the answer, respond with "I don't know"."""}
user_message = {"role": "user", "content": 
    """My robot spot_jr can't make left turns. When was it purchased and last serviced? What is the API that I can call?"""}

## Azure OpenAI API

In [ ]:
tools = [
  {
    "type": "function",
    "function": {
      "name": "get_robots",
      "description": "Get the name, kind, purchase date, and manufacturer of currently registered robots",
      "parameters": {
          "type": "object",
          "properties": {
              "name": {"type": "string", "description": "The name of the robot to get information about"},
              "kind": {"type": "string", "description": "the kind of the robot to get information about"},
          },
          "required": [],
      },
    },
  },
  {
    "type": "function",
    "function": {
      "name": "search_robot_manuals",
      "description": "Search the robot manuals for robot faults, troubleshooting guides, development API etc.",
      "parameters": {
          "type": "object",
          "properties": {
              "query": {"type": "string", "description": "The user's query for semantic and vector search"},
              "kind": {"type": "string", "description": "the kind of the robot to get information about"},
          },
          "required": ["query"],
      },
    },
  }
]

In [ ]:
from openai import AzureOpenAI

aoai = AzureOpenAI(api_version=aoai_api_version)
messages = [system_message, user_message]

num_turns = 0
while True:
  print_llm_messages(messages)
  response = aoai.chat.completions.create(
      model=aoai_deployment,
      messages=messages,
      tools=tools,
      tool_choice="auto", # let the model decide. "none" means don't call, or specify which tool to always call. 
  )
  result_message = response.choices[0].message
  tool_calls = result_message.tool_calls

  # no more tools to call
  if not tool_calls:
    print_llm_messages(result_message)
    break

  # doesn't make sense to call tools over and over
  if num_turns > 2:
    logging.warning("force terminated.")
    print_llm_messages(result_message)
    break

  # call tools, add tool results to messages
  result_message.content = "" if not result_message.content else result_message.content
  messages.append(result_message)
  for tool_call in tool_calls:
    if tool_call.function.name == "get_robots":
      args = json.loads(tool_call.function.arguments)
      function_result = get_robots(**args)
      messages.append({
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": tool_call.function.name,
        "content": function_result
      })
      num_turns += 1
    elif tool_call.function.name == "search_robot_manuals":
      args = json.loads(tool_call.function.arguments)
      function_result = search_robot_manuals(**args)
      messages.append({
        "tool_call_id": tool_call.id,
        "role": "tool",
        "name": tool_call.function.name,
        "content": function_result
      
      })
      num_turns += 1

### Debuggability

|Options|Pros|Cons|
|:-----|:---|:---|
|Python logging|<ul><li>With logging level set to INFO, http requests/responses are logged with headers.</li></ul>|<ul><li>Prompts are not logged automatically.</li><li>Format is hard to read.</li></ul>|
|Azure Monitor|<ul><li>Out of the box dashboard with metrics for tokens, requests etc.</li> <li>With diagnostics settings enabled on Azure OpenAI, http requests/responses are logged with headers.</li></ul>|<ul><li>Prompts are not automatically logged.</li></ul>|
|PPrint|<ul><li>Prompts can be manually logged in easy to read format.</li></ul>|<ul><li>Developer toil.</li></ul>|

## prompt flow with Azure OpenAI API

### Differences in code

-  Must write [flow.dag.yaml](./chat_flow/flow.dag.yaml).
-  Certain variables, such as [Azure OpenAI deployment name](./chat_flow/flow.dag.yaml#L24), cannot be loaded from .env unless you write your own code.
-  prompt flow tools must be annotated in certain ways. They are not the same as `tools` in OpenAI API. For example, here's a [prompt flow tool](./chat_flow/llm_tools.py)  that defines OpenAI `tools`.
-  Flow must run from the flow folder, which makes [importing existing Python files in other folders complicated](./chat_flow/agent_tool.py).
-  If you want to use `connections` in your non-prompt-flow code, you need to [translate](./chat_flow/agent_tool.py#L16) prompt flow `AzureOpenAIConnection` to openai `AzureOpenAI`.

### Differences in debuggability

-  With VSCode prompt flow extension, you can see the inputs/outputs of LLM calls automatically in the `prompt flow tab` without having to create any flow run.
-  You can also use `pf run create -f run.yaml --variant "${node.variant}"` to create a run for each variant, and then use `pf run visualize --names "run1,run2..."` to visualize multiple runs and compare their output in a nice table.



## LangChain Agent

In [ ]:
from langchain.chat_models import AzureChatOpenAI

from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.tools import  Tool, tool
from langchain_core.messages import HumanMessage, SystemMessage

In [ ]:
@tool("search_robot_manuals")
def search_robot_manuals_tool(query: str, kind: str = None) -> str:
  """Search the robot manuals for robot faults, troubleshooting guides, development API etc."""
  return search_robot_manuals(query, kind)

@tool("get_robots")
def get_robots_tool(name: str = None, kind: str = None) -> str:
  """Get info such as the name, kind, purchase date, and manufacturer of currently registered robots."""
  return get_robots(name, kind)

lc_tools = [search_robot_manuals_tool, get_robots_tool]

### Community and Composability

-  With the community OpenAPI agent, you can make LLM call any API with an OpenAPI spec, not just GET, but also any operations.
-  LangChain Agent is a model (an LLM). AgentExecutor is a chain(RunnableSerializable). So to use an existing agent as a tool, you can pass AgentExecutor.run method. Only the `run` method works, `invoke` doesn't. But `run` is deprecated.
-  There a bigger question of what is the right agent approach? When the agents perform completely different tasks in different domains, research from Microsoft AutoGen shows that having each agent specializing on a specific domain renders better results, even though the underlying model is the same.
    -  one agent with multiple tools?
    -  one orchestrator agent with multiple expert agents? 
    -  group of agents?

In [ ]:
from langchain.agents.agent_toolkits.openapi.spec import reduce_openapi_spec
from langchain.agents.agent_toolkits.openapi import planner
from langchain.requests import RequestsWrapper
import yaml

robotConfig = RobotConfig()
for kind in robotConfig.kinds:
  with open(kind['api_spec']) as f:
    raw_api_spec = yaml.load(f, Loader=yaml.Loader)
  api_spec = reduce_openapi_spec(raw_api_spec)
  requests_wrapper = RequestsWrapper()
  agent_executor: AgentExecutor = planner.create_openapi_agent(api_spec, requests_wrapper, lc_aoai)
  # AgentExecutor is a chain
  agent_tool = Tool.from_function(
    func=agent_executor.run,
    name=f"get_{kind['name']}_maintenance_records",
    description=f"Tool for accessing maintenance records for {kind['name']} robots.")
  lc_tools.append(agent_tool)

Start OpenAPI functions that expose maintenance data:

```sh
flask --app chat/functions/petoi.py run --port 3000
flask --app chat/functions/spot.py run --port 5000
```


In [ ]:
lc_aoai = AzureChatOpenAI(azure_deployment=aoai_deployment, openai_api_version=aoai_api_version)

prompt = ChatPromptTemplate.from_messages([
  SystemMessage(content=system_message['content']),
  HumanMessage(content=user_message['content']),
  MessagesPlaceholder(variable_name="agent_scratchpad")])

agent = create_openai_tools_agent(lc_aoai, lc_tools, prompt=prompt)
agent_executor = AgentExecutor(agent=agent, tools=lc_tools, verbose=True)
result = agent_executor({})
pprint.pprint(result)


### Differences in code

-  Must use `langchain.chat_models.AzureChatOpenAI`, although unlike `promptflow.connections.AzureOpenAIConnection`, LangChain does use the same OpenAI environment variables.
-  Must use LangChain `HumanMessage`, `SystemMessage` data types, and annotations such as `@tool`.
-  LangChain simplifies the code by automatically calling the tools, feed tools output back to LLM.
    -  note that it's still calling the tools one by one, it is parallizable, just not implemented that way yet.
-  good visibility with verbose=True, excellent in langsmith.

### Differences in debuggability

-  With LangSmith enabled by setting a few environment variables, all traces, including intermediate steps with inputs and outputs to LLMs, can be visualized in a nice UI.
-  Without LangSmith, set `verbose=True` in the `AgentExecutor` also enables easy-to-visualize logging in the console.

## OpenAI Assistants API


Not yet available in Azure OpenAI. Assistants will automatically call built-in tools such as Code Interpreter and Knowledge Retrieval, but will wait for the developer to make Function Calling. 